# Raster index

In [ ]:
%xmode minimal

import xarray as xr

import rasterix

xr.set_options(display_expand_indexes=True)

## Example with rectilinear and no rotation affine transform

Both x and y coordinates are 1-dimensional.

In [ ]:
source = "/vsicurl/https://noaadata.apps.nsidc.org/NOAA/G02135/south/daily/geotiff/2024/01_Jan/S_20240101_concentration_v4.0.tif"

### Load and inspect the datasets, with and without `RasterIndex`.

In [ ]:
da_no_raster_index = xr.open_dataarray(source, engine="rasterio")
da_raster_index = rasterix.assign_index(da_no_raster_index)

Let's compare the coordinates for the DataArrays with and without RasterIndex

In [ ]:
da_no_raster_index.x

By contrast, these are lazy!

In [ ]:
da_raster_index.x

These differences are viewable in the repr. Note the `PandasIndex` type under "Indexes".

In [ ]:
da_no_raster_index

 The repr below shows a few values for each coordinate (those have been computed on-the-fly) but clicking on the database icon doesn't show any value in the spatial coordinate data reprs.

In [ ]:
da_raster_index

### Positional Indexing (`isel`)

#### Slicing both x and y

In [ ]:
da_sliced = da_raster_index.isel(x=slice(1, 4), y=slice(None, None, 2))
da_sliced

Slicing keeps both coordinates lazy (it computes a new affine transform):

In [ ]:
print(da_sliced.xindexes["x"])
print(da_sliced.xindexes["y"])

#### Outer indexing with arbitrary array values*

In [ ]:
da_outer = da_raster_index.isel(x=[0, 2, 4], y=[0, 0, 1])
da_outer

We cannot compute a new affine transform given arbitrary array positions. To allow further data selection, pandas indexes are _automatically_ created for indexed spatial coordinates:

In [ ]:
print(da_outer.xindexes["x"])
print(da_outer.xindexes["y"])

#### Basic indexing with scalars

In [ ]:
da_scalar = da_raster_index.isel(x=0, y=1)
da_scalar

In [ ]:
da_xscalar = da_raster_index.isel(x=0)
da_xscalar

**FIXME** The RasterIndex should be preserved in case of partial dimension reduction.

In [ ]:
# da_xscalar.xindexes["y"]  # should return an index

#### Vectorized (fancy) indexing

Indexing the spatial coordinates with Xarray `Variable` objects returns a `RasterIndex` (wrapping `PandasIndex`) for 1-dimensional variables and no index for scalar or n-dimensional variables.

In [ ]:
da_points = da_raster_index.isel(x=xr.Variable("z", [0, 1]), y=xr.Variable("z", [1, 1]))
da_points

In [ ]:
da_points2d = da_raster_index.isel(
    x=xr.Variable(("u", "v"), [[0, 1], [2, 3]]),
    y=xr.Variable(("u", "v"), [[1, 1], [2, 2]]),
)
da_points2d

### Label Indexing (`sel`)

Label-based indexing also preserves the RasterIndex where possible.

In [ ]:
da_raster_index.sel(x=slice(-2e6, 2e6), y=slice(4e6, -2e6))

### Equality

`equals` compares variable values without relying on Xarray coordinate indexes. Both dataarrays should thus be equal.

In [ ]:
da_raster_index.equals(da_no_raster_index)

### Alignment

Xarray alignment relies on Xarray coordinate indexes. Trying to align both datasets fails here since they each have different index types.

In [ ]:
da_raster_index + da_no_raster_index

Implementing joins is on the roadmap.

In [ ]:
xr.align(da_raster_index, da_raster_index.isel(x=slice(10), y=slice(10)), join="inner")